pmgus_vbeta.ipynb

In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.colheader_justify', 'left')  # Left-align column headers
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
from datetime import datetime, timedelta
#
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import pytz
from tqdm import tqdm  # Visualize loop progress
from sklearn.linear_model import LinearRegression
from tenacity import retry, stop_after_attempt, wait_fixed
import openpyxl
import os
import requests
import yfinance as yf

#
from dotenv import load_dotenv

data import (manual) TradingView

In [2]:
# base file path and file path setup
base_dir_path = '/Users/sudz4/Desktop/SPS_local/sps/x_pre_market_gap_up_screener/' 
tv_prefix = 'tv_screen_gap-up_'
file_type_csv = '.csv'
#####---------------------#####
# screen_date = '2024-12-17'
screen_date = '2024-12-18'

#####---------------------#####
tv_filename = base_dir_path + tv_prefix + screen_date + file_type_csv

# READ
trading_view_df = pd.read_csv(tv_filename)

# PRINT
print(tv_filename)
print(len(trading_view_df))
# print first 5 ticker Symbols only
print(trading_view_df['Symbol'].head(5))

/Users/sudz4/Desktop/SPS_local/sps/x_pre_market_gap_up_screener/tv_screen_gap-up_2024-12-18.csv
1820
0     NVDA
1     AMZN
2     META
3    BRK.B
4      TSM
Name: Symbol, dtype: object


setup market cap category groups, criteria configurations, and conditional filtering.

In [3]:
# CREATE CATEGORIES FOR MARKET CAP
def categorize_market_cap(df):
    # categorize stocks groups by market cap
    df['Market capitalization'] = pd.to_numeric(df['Market capitalization'], errors='coerce')
    conditions = [
        (df['Market capitalization'] >= 200_000_000_000),  # Titans
        (df['Market capitalization'] >= 10_000_000_000) & (df['Market capitalization'] < 200_000_000_000),  # Large caps
        (df['Market capitalization'] >= 2_000_000_000) & (df['Market capitalization'] < 10_000_000_000),  # Mid caps
        (df['Market capitalization'] >= 300_000_000) & (df['Market capitalization'] < 2_000_000_000),  # Small caps
        (df['Market capitalization'] > 50_000_000) & (df['Market capitalization'] < 300_000_000),  # Micro caps
        (df['Market capitalization'] <= 50_000_000)  # Shrimp
    ]
    # marekt cap categories list
    categories = ['Titans', 'Large caps', 'Mid caps', 'Small caps', 'Micro caps', 'Shrimp']
    df['marketCapType'] = np.select(conditions, categories, default='Undefined')
    return df
# execute categorization
category_setup_df = categorize_market_cap(trading_view_df).copy()
# drop Undefined marketCapType
category_setup_df = category_setup_df[category_setup_df['marketCapType'] != 'Undefined']

# convert necessary columns to numeric
def convert_columns_to_numeric(df, columns):
    """Convert specified columns to numeric types."""
    for col in columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')
    return df

# list of columns to convert
numeric_columns = [
    'Market capitalization', 'Float shares outstanding', 'Relative Volume 1 day',
    'Relative Volume at Time', 'Pre-market Change %', 'Pre-market Gap %',
    'Price', 'Volume Weighted Average Price 1 day', 'Volatility 1 day',
    'Volatility 1 week', 'Volatility 1 month', 'Pre-market Volume'
]

# apply conversion
category_setup_df = convert_columns_to_numeric(category_setup_df, numeric_columns)

# CRITERIA CONFIGURATION FOR EACH MARKET CAP CATEGORY
criteria_config = {
    "Titans": {
        "pre_market_change_pct_threshold": 0.002,  # 0.2% for Titans
        "float_shares_outstanding_threshold": 1_000_000_000,  # 1 billion shares
        "relative_volume_threshold": 1.2,
        "relative_volume_at_time_threshold": 0.03,
        "pre_market_gap_percentage_threshold": 0.001,  # 0.1%
        "pre_market_vwap_drawdown_threshold": 0.003,  # 0.3% drawdown from VWAP
        "pre_market_volume_threshold": 50_000  # Minimum pre-market volume
    },
    "Large caps": {
        "pre_market_change_pct_threshold": 0.005,  # 0.5% for Large caps
        "float_shares_outstanding_threshold": 200000000,  # 200 million shares
        "relative_volume_threshold": 1.3,  # More inclusive
        "relative_volume_at_time_threshold": 0.04,  # More inclusive
        "pre_market_gap_percentage_threshold": 0.005,  # 0.5%
        "pre_market_vwap_drawdown_threshold": 0.004,  # 0.4% drawdown from VWAP
        "pre_market_volume_threshold": 50000  # Minimum pre-market volume
    },
    "Midlers": { 
        "pre_market_change_pct_threshold": 0.02,  # 2% for Midlers 
        "float_shares_outstanding_threshold": 50000000,  # 50 million shares
        "relative_volume_threshold": 1.3,
        "relative_volume_at_time_threshold": 0.05,
        "pre_market_gap_percentage_threshold": 0.02,
        "pre_market_vwap_drawdown_threshold": 0.005,  # 0.5% drawdown from VWAP
        "pre_market_volume_threshold": 50000  # Minimum pre-market volume
    },
    "Small caps": {
        "pre_market_change_pct_threshold": 0.03,  # 3% for Small caps
        "float_shares_outstanding_threshold": 20000000,  # 20 million shares
        "relative_volume_threshold": 1.2,
        "relative_volume_at_time_threshold": 0.05,
        "pre_market_gap_percentage_threshold": 0.03,
        "pre_market_vwap_drawdown_threshold": 0.006,  # 0.6% drawdown from VWAP
        "pre_market_volume_threshold": 50000  # Minimum pre-market volume
    },
    "Micro caps": {
        "pre_market_change_pct_threshold": 0.04,  # 4% for Micro caps
        "float_shares_outstanding_threshold": 5000000,  # 5 million shares
        "relative_volume_threshold": 1.1,
        "relative_volume_at_time_threshold": 0.05,
        "pre_market_gap_percentage_threshold": 0.04,
        "pre_market_vwap_drawdown_threshold": 0.007,  # 0.7% drawdown from VWAP
        "pre_market_volume_threshold": 50000  # Minimum pre-market volume
    },
    "Shrimp": {
        "pre_market_change_pct_threshold": 0.05,  # 5% for Shrimp
        "float_shares_outstanding_threshold": 1000000,  # 1 million shares
        "relative_volume_threshold": 1.0,
        "relative_volume_at_time_threshold": 0.05,
        "pre_market_gap_percentage_threshold": 0.05,
        "pre_market_vwap_drawdown_threshold": 0.008, # 0.8% drawdown from VWAP
        "pre_market_volume_threshold": 50000  # Minimum pre-market volume
    }
}

# FILTER STOCKS BASED ON CONFIGURATION CRITERIA - STAGING
def filter_stocks(df, config):
    # stock filtering conditions >=< based on criteria
    conditions = (
        (df['Pre-market Change %'] >= config.get('pre_market_change_pct_threshold', 0)) &
        (df['Float shares outstanding'] <= config.get('float_shares_outstanding_threshold', float('inf'))) &
        (df['Relative Volume 1 day'] >= config.get('relative_volume_threshold', 0)) &
        (df['Relative Volume at Time'] >= config.get('relative_volume_at_time_threshold', 0)) &
        (df['Pre-market Gap %'] >= config.get('pre_market_gap_percentage_threshold', 0)) &
        (df['Price'] >= df['Volume Weighted Average Price 1 day'] * (1 - config.get('pre_market_vwap_drawdown_threshold', 0))) &
        (df['Volatility 1 day'] >= df['Volatility 1 week']) &
        (df['Volatility 1 day'] >= df['Volatility 1 month']) &
        (df['Pre-market Volume'] >= config.get('pre_market_volume_threshold', 0))
    )
    return df[conditions]

# SCREEN STOCKS BY CATEGORY
def screen_stocks_by_category(df, category):
    """Filter stocks in a category using predefined criteria."""
    config = criteria_config.get(category, {})
    filtered_df = filter_stocks(df, config)
    return filtered_df

# EXECUTE KEY SCREENING FUNCTION AND CREATE DATAFRAME
smash_df = pd.DataFrame()
categories = category_setup_df['marketCapType'].unique()

for category in categories:
    category_df = category_setup_df[category_setup_df['marketCapType'] == category]
    gap_up_stage_df = screen_stocks_by_category(category_df, category)
    smash_df = pd.concat([smash_df, gap_up_stage_df], ignore_index=True)

# rendered column list and ordering
cols_list = [
    'Symbol', 
    'Description', 
    'marketCapType', 
    'Pre-market Change %', 
    'Pre-market Gap %', 
    'marketCapType',
    'Market capitalization',
    'Price', 
    'Pre-market Open', 
    'Industry', 
    'Index', 
    'Sector', 
    'Exchange',
    'Recent earnings date', 
    'Upcoming earnings date', 
    'Float shares outstanding', 
    'Average Volume 10 days',
    'Average Volume 30 days', 
    'Average Volume 90 days',
    'Relative Volume 1 day', 
    'Relative Volume 5 minutes', 
    'Relative Volume 30 minutes', 
    'Relative Volume at Time', 
    'Analyst Rating',
    'Technical Rating 5 minutes'
]

# filter columns to only include those present in the DataFrame
existing_cols = [col for col in cols_list if col in smash_df.columns]
smash_df = smash_df[existing_cols]

# sort and reset index
smash_df = smash_df.sort_values(
    by=['Pre-market Change %', 'Price'],
    ascending=[False, False]).reset_index(drop=True)

print(f"{smash_df.shape[0]} stocks found.")

54 stocks found.


quick and dirty static screen

TODO****
think abdout adding float???

In [4]:
# quick and dirty filter for high-probability morning momentum trades (Re: Pre-market Gap Up)
def quick_dirty_filter(df):
    return df[
        # strong pre-market movement but not overextended
        (
            (df['Pre-market Change %'] > 3.0) |
            ((df['Pre-market Change %'] > 1.5) & (df['Relative Volume 5 minutes'] > 5.0))
        ) &
        # volume validation
        (df['Relative Volume 5 minutes'] > 2.0) &
        # industry/sector grouping
        (df.groupby('Industry')['Pre-market Change %'].transform('count') > 1)
            ].sort_values('Pre-market Change %', ascending=False)

In [5]:
# EXECUTE quick and dirty filter && CREATE a df and list
smash_qd_df = quick_dirty_filter(smash_df)
smash_qd_list = smash_qd_df['Symbol'].to_list()

# PRINT
print(f'{len(smash_qd_df)} stocks returned from the quick and dirty filter.')
# print(smash_qd_df.columns)
print(smash_qd_list)
display(smash_qd_df)

10 stocks returned from the quick and dirty filter.
['ARQQ', 'OKLO', 'JBL', 'SDRL', 'WK', 'LTH', 'RIG', 'GLNG', 'STEP', 'CRS']


,Symbol,Description,marketCapType,Pre-market Change %,Pre-market Gap %,marketCapType,Market capitalization,Price,Pre-market Open,Industry,Index,Sector,Exchange,Recent earnings date,Upcoming earnings date,Float shares outstanding,Average Volume 10 days,Average Volume 30 days,Average Volume 90 days,Relative Volume 1 day,Relative Volume 5 minutes,Relative Volume 30 minutes,Relative Volume at Time,Analyst Rating,Technical Rating 5 minutes
1,ARQQ,Arqit Quantum Inc.,Small caps,14.787879,1.333333,Small caps,4.126323e+08,33.00,33.44,Packaged software,"NASDAQ Composite, NASDAQ Computer",Technology services,NASDAQ,2024-12-05,2025-05-08,4.726850e+06,1210359.1,8.370858e+05,3.468775e+05,2.412323,3.170717,1.475686,2.403656,Strong buy,Strong buy
2,OKLO,Oklo Inc.,Mid caps,12.984293,0.418848,Mid caps,2.332039e+09,19.10,19.18,Industrial machinery,NaN,Producer manufacturing,NYSE,2024-11-14,2025-02-26,7.226545e+07,8344330.0,1.442053e+07,1.206641e+07,0.877310,3.338028,3.189939,0.826468,Buy,Buy
4,JBL,Jabil Inc.,Large caps,10.107495,4.844730,Large caps,1.496240e+10,133.96,140.45,Industrial machinery,"S&P 500, S&P 500 Information Technology, Russell 3000, Russell 1000",Producer manufacturing,NYSE,2024-12-18,2025-03-13,1.069729e+08,978645.9,1.084584e+06,1.303804e+06,2.056526,10.920616,5.353767,2.205496,Buy,Buy
5,SDRL,Seadrill Limited,Mid caps,3.430716,2.573037,Mid caps,2.559307e+09,37.31,38.27,Contract drilling,"Russell 2000, Russell 3000, Mini-Russell 2000",Industrial services,NYSE,2024-11-12,2025-02-27,6.518601e+07,774313.9,8.967726e+05,1.073933e+06,1.169186,9.910425,4.185289,1.172491,Strong buy,Buy
6,WK,Workiva Inc.,Mid caps,2.799890,2.022143,Mid caps,6.062223e+09,109.29,111.50,Information technology services,"Russell 2000, ISE CTA Cloud Computing, Nasdaq US Small Cap Growth, Russell 3000, Mini-Russell 2000",Technology services,NYSE,2024-11-06,2025-02-25,4.869814e+07,418517.0,4.540817e+05,3.610348e+05,1.158978,6.462511,3.999959,1.351849,Strong buy,Neutral
7,LTH,"Life Time Group Holdings, Inc.",Mid caps,2.606838,2.606838,Mid caps,4.843159e+09,23.40,24.01,Other consumer services,"Russell 2000, Nasdaq US Small Cap Growth, Russell 3000, Mini-Russell 2000",Consumer services,NYSE,2024-10-24,2025-03-06,8.053595e+07,1053998.1,1.096782e+06,1.463819e+06,1.290267,7.515955,7.290542,1.299331,Buy,Neutral
9,RIG,Transocean Ltd (Switzerland),Mid caps,2.449591,1.089918,Mid caps,3.214199e+09,3.67,3.71,Contract drilling,"Russell 2000, Russell 3000, Mini-Russell 2000, PHLX Oil Service Sector",Industrial services,NYSE,2024-10-30,2025-02-24,7.311063e+08,23246027.4,1.986339e+07,1.964228e+07,1.109158,5.565914,2.105223,1.063373,Neutral,Buy
11,GLNG,Golar LNG Limited,Mid caps,1.824645,1.824645,Mid caps,4.412723e+09,42.20,42.97,Marine shipping,"NASDAQ Composite, Russell 2000, Russell 3000, NASDAQ Industrials, Mini-Russell 2000, PHLX Oil Service Sector",Transportation,NASDAQ,2024-11-12,2025-03-04,9.322284e+07,1692807.9,1.519232e+06,1.237711e+06,1.249986,7.030219,5.450027,0.876356,Strong buy,Sell
13,STEP,StepStone Group Inc.,Mid caps,1.765650,1.765650,Mid caps,7.174673e+09,62.30,63.40,Investment managers,"NASDAQ Composite, Russell 2000, Nasdaq US Small Cap Growth, Russell 3000, Mini-Russell 2000, NASDAQ Real Estate and Other Financial Services",Finance,NASDAQ,2024-11-07,2025-02-11,6.259290e+07,865515.1,7.021475e+05,6.413104e+05,0.722687,5.775957,5.254485,0.608899,Buy,Buy
14,CRS,Carpenter Technology Corporation,Mid caps,1.640009,1.130042,Mid caps,8.601229e+09,172.56,174.51,Other metals/Minerals,"Russell 2000, Nasdaq US Mid Cap Growth, Russell 3000, Mini-Russell 2000",Non-energy minerals,NYSE,2024-10-24,2025-01-30,4.729274e+07,704010.6,6.274774e+05,5.715210e+05,1.374884,6.003538,3.136441,1.550081,Strong buy,Buy


save stock(s) results (output) to .csv

In [ ]:
# string object for quick and dirty results .csv
smash_qd_results_filename = (f"{base_dir_path}smash_quick_dirty_results_{screen_date}{file_type_csv}")
print(smash_qd_results_filename)

# SAVE TO CSV
smash_qd_df.to_csv(smash_qd_results_filename, index=False)

/Users/sudz4/Desktop/SPS_local/sps/x_pre_market_gap_up_screener/smash_quick_dirty_results_2024-12-18.csv


In [10]:
print(len(smash_qd_list))
print(len(smash_qd_df))
print(smash_qd_list)

10
10
['ARQQ', 'OKLO', 'JBL', 'SDRL', 'WK', 'LTH', 'RIG', 'GLNG', 'STEP', 'CRS']
